In [2]:
# Dependencies
import tweepy
import numpy as np

# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Twitter API Keys
from config import (consumer_key, 
                    consumer_secret, 
                    access_token, 
                    access_token_secret)

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [3]:
# Target Search Term
target_terms = ("@SouthwestAir", "@AmericanAir", "@SpiritAirlines",
                "@Virginatlantic", "@Delta", "@AlaskaAir", "@KLM")

# "Real Person" Filters
min_tweets = 5
max_tweets = 10000
max_followers = 2500
max_following = 2500
lang = "en"

# Array to hold sentiment
sentiment_array = []

# Loop through all target users
for target in target_terms:

    # Variable for holding the oldest tweet
    oldest_tweet = None

    # Variables for holding sentiments
    compound_list = []
    positive_list = []
    negative_list = []
    neutral_list = []

    # make multiple requests
    for x in range(2):

        # Run search around each tweet
        public_tweets = api.search(
            target, count=10, result_type="recent", max_id=oldest_tweet)

        # Loop through all tweets
        for tweet in public_tweets["statuses"]:

            # Use filters to check if user meets conditions
            if (tweet["user"]["followers_count"] < max_followers
                and tweet["user"]["statuses_count"] > min_tweets
                and tweet["user"]["statuses_count"] < max_tweets
                and tweet["user"]["friends_count"] < max_following
                and tweet["user"]["lang"] == lang):

                # Run Vader Analysis on each tweet
                results = analyzer.polarity_scores(tweet["text"])
                compound = results["compound"]
                pos = results["pos"]
                neu = results["neu"]
                neg = results["neg"]

                # Add each value to the appropriate array
                compound_list.append(compound)
                positive_list.append(pos)
                negative_list.append(neg)
                neutral_list.append(neu)
                
        # Set the new oldest_tweet value
        oldest_tweet = int(tweet["id_str"]) - 1

    # Store the Average Sentiments
    sentiment = {
        "User": target,
        "Compound": np.mean(compound_list),
        "Positive": np.mean(positive_list),
        "Neutral": np.mean(negative_list),
        "Negative": np.mean(neutral_list),
        "Tweet Count": len(compound_list)
    }

    # Print the Sentiments
    print(sentiment)
    print()

{'User': '@SouthwestAir', 'Compound': 0.16962222222222223, 'Positive': 0.11299999999999999, 'Neutral': 0.035, 'Negative': 0.852, 'Tweet Count': 9}

{'User': '@AmericanAir', 'Compound': -0.10311, 'Positive': 0.0, 'Neutral': 0.0366, 'Negative': 0.9634, 'Tweet Count': 10}

{'User': '@SpiritAirlines', 'Compound': 0.12732500000000002, 'Positive': 0.14033333333333334, 'Neutral': 0.10983333333333332, 'Negative': 0.74975, 'Tweet Count': 12}

{'User': '@Virginatlantic', 'Compound': 0.25101428571428575, 'Positive': 0.1702857142857143, 'Neutral': 0.07842857142857143, 'Negative': 0.7512857142857143, 'Tweet Count': 7}

{'User': '@Delta', 'Compound': 0.10033636363636363, 'Positive': 0.09209090909090908, 'Neutral': 0.03663636363636364, 'Negative': 0.8712727272727272, 'Tweet Count': 11}

{'User': '@AlaskaAir', 'Compound': 0.47532307692307696, 'Positive': 0.1877692307692308, 'Neutral': 0.014846153846153846, 'Negative': 0.7973846153846155, 'Tweet Count': 13}

{'User': '@KLM', 'Compound': 0.1902, 'Positi